<a href="https://colab.research.google.com/github/FrancoPalavicinoG/cellia/blob/main/notebooks/04_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Train, Val & Test Datatsets from Google Drive

Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [2]:
# Core
import pandas as pd
import numpy as np

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, classification_report

# Explainability
import shap
import matplotlib.pyplot as plt
import seaborn as sns

Load Datasets

In [3]:
input_path = "/content/drive/MyDrive/cellia_drive/Datasets/"

Load features

In [4]:
X_train = pd.read_csv(input_path + "X_train_res.csv")
X_val   = pd.read_csv(input_path + "X_val.csv")
X_test  = pd.read_csv(input_path + "X_test.csv")

Load labels

In [5]:
y_train = pd.read_csv(input_path + "y_train_res.csv").squeeze()
y_val   = pd.read_csv(input_path + "y_val.csv").squeeze()
y_test  = pd.read_csv(input_path + "y_test.csv").squeeze()

In [6]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6580, 25) (6580,)
(782, 25) (782,)
(783, 25) (783,)


### Evaluation function

In [7]:
def evaluate_model(model, X, y_true, dataset_name="Validation"):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    roc_auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    print(f"{dataset_name} Results:")
    print(f"ROC-AUC: {roc_auc:.3f} | F1: {f1:.3f} | Precision: {precision:.3f} | Recall: {recall:.3f}")
    print("-" * 60)

    return {"roc_auc": roc_auc, "f1": f1, "precision": precision, "recall": recall}

## Model Training

 Reindex val and test columns

In [8]:
# features = X_train.columns.tolist()

# X_val = X_val.reindex(columns=features, fill_value=0)
# X_test = X_test.reindex(columns=features, fill_value=0)

# assert all(X_train.columns == X_val.columns)
# assert all(X_train.columns == X_test.columns)

### Logistic Regression (Baseline)

In [9]:
log_reg = LogisticRegression(
    max_iter=500,          # iteration numbers
    random_state=42,
    class_weight='balanced',  # class balancing
    C=1.0,                 # regularization
    penalty='l2',          # penalty type: 'l1', 'l2', 'elasticnet'
    solver='liblinear'     # solver compatible with the penalty
)


### Random Forest

In [10]:
rf = RandomForestClassifier(
    n_estimators=300,      # number of trees
    max_depth=10,          # maximum depth of each tree
    min_samples_split=2,   # minimum samples to split a node
    min_samples_leaf=1,    # minimum samples per leaf
    max_features='sqrt',   # number of features to consider per split
    random_state=42,
    class_weight='balanced'
)


### XGBoost

In [11]:
# xgb = XGBClassifier(
#     n_estimators=300,
#     learning_rate=0.05,
#     max_depth=6,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     eval_metric='logloss'
# )
# xgb.fit(X_train, y_train)

# evaluate_model(xgb, X_val, y_val, "Validation (XGBoost)")

### LightGBM

In [12]:
lgbm = LGBMClassifier(
    n_estimators=300,      # number of iterations
    max_depth=6,           # maximum tree depth
    num_leaves=31,         # leaves per tree
    learning_rate=0.05,    # learning rate
    min_child_samples=20,  # minimum samples per leaf node
    subsample=0.8,         # fraction of data per tree
    colsample_bytree=0.8,  # fraction of features per tree
    random_state=42,
    scale_pos_weight=1     # class balancing if not using SMOTE
)

### MLP Tabular

In [13]:
# mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
# mlp.fit(X_train, y_train)

# evaluate_model(mlp, X_val, y_val, "Validation (MLP)")

### Train models

In [14]:
models = {"Logistic Regression": log_reg,
          "Random Forest": rf,
          "LightGBM": lgbm}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

Training Logistic Regression...
Validation (Logistic Regression) Results:
ROC-AUC: 0.852 | F1: 0.401 | Precision: 0.272 | Recall: 0.766
------------------------------------------------------------
Training Random Forest...
Validation (Random Forest) Results:
ROC-AUC: 0.821 | F1: 0.421 | Precision: 0.354 | Recall: 0.519
------------------------------------------------------------
Training LightGBM...
[LightGBM] [Info] Number of positive: 3290, number of negative: 3290
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001657 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6375
[LightGBM] [Info] Number of data points in the train set: 6580, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

### Compare models

In [15]:
models = {
    "Logistic Regression": log_reg,
    "Random Forest": rf,
    "LightGBM": lgbm,
}

results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)

Validation (Logistic Regression) Results:
ROC-AUC: 0.852 | F1: 0.401 | Precision: 0.272 | Recall: 0.766
------------------------------------------------------------
Validation (Random Forest) Results:
ROC-AUC: 0.821 | F1: 0.421 | Precision: 0.354 | Recall: 0.519
------------------------------------------------------------
Validation (LightGBM) Results:
ROC-AUC: 0.810 | F1: 0.383 | Precision: 0.422 | Recall: 0.351
------------------------------------------------------------


,roc_auc,f1,precision,recall
Logistic Regression,0.851727,0.401361,0.271889,0.766234
Random Forest,0.821092,0.421053,0.353982,0.519481
LightGBM,0.810463,0.382979,0.421875,0.350649


### Hyperparameter Tuning using RandomizedSearchCV

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from scipy.stats import uniform, randint
import numpy as np

# ---------------------------
# Logistic Regression
# ---------------------------
logreg = LogisticRegression(max_iter=1000, random_state=42)

logreg_param_dist = {
    'C': uniform(0.01, 10),
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

logreg_search = RandomizedSearchCV(
    logreg,
    param_distributions=logreg_param_dist,
    n_iter=20,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

logreg_search.fit(X_train, y_train)
print("Best Logistic Regression params:", logreg_search.best_params_)
print("Best F1:", logreg_search.best_score_)

# ---------------------------
# Random Forest
# ---------------------------
rf = RandomForestClassifier(random_state=42)

rf_param_dist = {
    'n_estimators': randint(200, 1000),
    'max_depth': randint(5, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

rf_search = RandomizedSearchCV(
    rf,
    param_distributions=rf_param_dist,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)
print("Best Random Forest params:", rf_search.best_params_)
print("Best F1:", rf_search.best_score_)

# ---------------------------
# LightGBM
# ---------------------------
lgbm = LGBMClassifier(random_state=42)

lgbm_param_dist = {
    'num_leaves': randint(20, 50),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.01, 0.1),
    'n_estimators': randint(200, 1000),
    'min_child_samples': randint(10, 50),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'class_weight': [None, 'balanced']
}

lgbm_search = RandomizedSearchCV(
    lgbm,
    param_distributions=lgbm_param_dist,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

lgbm_search.fit(X_train, y_train)
print("Best LightGBM params:", lgbm_search.best_params_)
print("Best F1:", lgbm_search.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Logistic Regression params: {'C': np.float64(4.5706998421703595), 'class_weight': None, 'penalty': 'l2', 'solver': 'saga'}
Best F1: 0.7985362538992313
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Random Forest params: {'class_weight': None, 'max_depth': 16, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 947}
Best F1: 0.928177405318125
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Number of positive: 3290, number of negative: 3290
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6375
[LightGBM] [Info] Number of data points in the train set: 6580, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f